In [71]:
import pickle
#from loadData import dataLoader
import tensorflow as tf
from CalculateEER import SKGetEER as GetEER


In [72]:
def onehot(x,numlabels):
    t = [0 for i in range(numlabels)]
    t[x-1] = 1
    return t

In [73]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
train_data = pickle.load(open('/Users/chaitanya/Documents/python/keystrokes/data_augmentation/greyc1/train_greyc1_2x.pickle','rb'))
test_data = pickle.load(open("/Users/chaitanya/Documents/python/keystrokes/keystrokes3/cnnNN/greyc1/PCA_GREYC1_test.pickle", "rb"))


In [74]:
X_train = train_data['data']
X_train = np.array(X_train).reshape(len(X_train), 1, 28, 1)
y_train = train_data['labels']
#y_train = [onehot(x,100) for x in train_data['labels']]
X_test = np.array(test_data['features']).reshape(len(test_data['features']), 1, 28, 1)
y_test = [onehot(x,100) for x in test_data['labels']]
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1)
print([len(x) for x in [X_test, X_val, y_test, y_val, X_train, y_train]])

[1012, 1013, 1012, 1013, 8459, 8459]


In [75]:
import numpy as np
np.shape(X)


(20241, 1, 32, 1)

In [76]:
tf.reset_default_graph()
inp = tf.placeholder(tf.float32, shape=[None, 1, 28, 1], name = "inp")
out = tf.placeholder(tf.float32, shape = [None,100], name = "out")


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([1, 5, 1, 20])
b_conv1 = bias_variable([20])

h_conv1 = tf.nn.relu(conv2d(inp, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([1, 5, 20, 128])
b_conv2 = bias_variable([128])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

print(h_pool2.shape)
W_fc1 = weight_variable([1 * 7 * 128, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 1*7*128])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 100])
b_fc2 = bias_variable([100])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
y_ = tf.nn.softmax(y_conv)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=out, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(out, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

(?, 1, 7, 128)


In [56]:
X.shape[0]

20241

In [77]:
with tf.Session() as sess:
    try:
        sess.run(tf.global_variables_initializer())
        batchSize = 64
        for epoch in range(500):
            j = -1
            i = 0
            while i < X_train.shape[0]:
                j += 1
                batch = X_train[i: i + batchSize]
                batch_labels = y_train[i: i + batchSize]
                i = i + batchSize
                #print(h_pool2.eval(feed_dict = {inp: batch, keep_prob: 1.0}).shape)
                sess.run(train_step, feed_dict={inp: batch, out: batch_labels, keep_prob: 0.5})
                if not j%50:
                    #print("here")
                    train_accuracy = accuracy.eval(feed_dict={inp: batch, out: batch_labels, keep_prob: 1.0})
                    train_out = sess.run(y_, feed_dict={inp: batch, out: batch_labels, keep_prob: 1.0})
                    valid_accuracy = accuracy.eval(feed_dict={inp: X_val, out: y_val, keep_prob: 1.0})
                    valid_out = sess.run(y_, feed_dict={inp: X_val, out: y_val, keep_prob: 1.0})
                    #print(valid_out[0])
                    test_accuracy = accuracy.eval(feed_dict={inp: X_test, out: y_test, keep_prob: 1.0})
                    test_out = sess.run(y_, feed_dict={inp: X_test, out: y_test, keep_prob: 1.0})
                    TEER = GetEER(np.array(test_out), np.array(y_test))
                    VEER = GetEER(np.array(valid_out), np.array(y_val))
                    print(epoch, j, train_accuracy, valid_accuracy, test_accuracy, VEER, TEER)
    except Exception as e:
        print(e)
        d = {}
        d['cnn'] = sess.run(y_, feed_dict={inp: data.data, out: data.data_labels, keep_prob: 1.0})
        d['y'] = data.data_labels
        pickle.dump(d, open("./Data/CNNGREYC1.pickle", "wb"))


0 0 0.0 0.009871668 0.009881423 0.4981901941428101 0.4875124875124875
0 50 0.0 0.016781837 0.01778656 0.46859421734795614 0.45991983967935873
0 100 0.0 0.04047384 0.040513836 0.43725099601593626 0.4272818455366098
1 0 0.0 0.041461006 0.03952569 0.42445328031809143 0.4121756487025948
1 50 0.0 0.042448174 0.052371543 0.38877755511022044 0.388
1 100 0.0 0.07798618 0.0770751 0.3715139442231076 0.3592039800995025
2 0 0.09375 0.078973345 0.09090909 0.35692921236291125 0.3433734939759036
2 50 0.046875 0.083909184 0.089920945 0.33134328358208953 0.328
2 100 0.0 0.12142152 0.12252964 0.3137450199203187 0.29810568295114653
3 0 0.484375 0.14215203 0.14426878 0.2912912912912913 0.288
3 50 0.046875 0.11253702 0.12944664 0.27976190476190477 0.2754491017964072
3 100 0.0 0.1697927 0.17588933 0.2634194831013916 0.2550403225806452
4 0 0.71875 0.1816387 0.17786561 0.25124626121635096 0.24373119358074222
4 50 0.140625 0.13524185 0.1521739 0.237 0.23123123123123124
4 100 0.0 0.21618953 0.21442688 0.2308457

37 100 0.84375 0.6841066 0.6966403 0.07447864945382324 0.08054856869086117
38 0 0.875 0.68904245 0.7035573 0.07157057654075547 0.08265460933445123
38 50 0.921875 0.6791708 0.6936759 0.07064676616915423 0.08408192597915919
38 100 0.84375 0.68904245 0.7005929 0.07263681592039802 0.08111749910168882
39 0 0.875 0.69792694 0.7075099 0.07028112449799197 0.0818062043358486
39 50 0.890625 0.6821323 0.7005929 0.07064676616915423 0.07980995728031301
39 100 0.875 0.69200397 0.7094862 0.07278165503489531 0.0765061684034016
40 0 0.890625 0.6939783 0.7094862 0.07157057654075547 0.08119734898391025
40 50 0.90625 0.6880553 0.70454544 0.06979062811565304 0.08301393380444763
40 100 0.875 0.69693977 0.71343875 0.07064676616915423 0.07926098934004072
41 0 0.890625 0.69693977 0.71146244 0.07014028056112225 0.08128718010140935
41 50 0.921875 0.68904245 0.708498 0.06986027944111776 0.08183632734530938
41 100 0.84375 0.69693977 0.708498 0.07064676616915423 0.07628658122729269
42 0 0.890625 0.6999013 0.7164032

74 100 0.953125 0.7769003 0.7786561 0.05018596627678563 0.06338084401325508
75 0 0.96875 0.7709773 0.7826087 0.047194551636802376 0.06274900398406374
75 50 1.0 0.7680158 0.770751 0.04933839879545704 0.06268215754381762
75 100 0.921875 0.78183615 0.7756917 0.050086252455452854 0.06003713019523296
76 0 0.96875 0.77788746 0.7835968 0.04879994416026005 0.06069651741293532
76 50 1.0 0.7739388 0.7687747 0.04941816985252326 0.060116980077454384
76 100 0.9375 0.77887464 0.78162056 0.05092384855464816 0.061691542288557215
77 0 0.96875 0.7769003 0.7855731 0.046646125619472116 0.06171397772188286
77 50 1.0 0.7719645 0.7766798 0.04570881569894403 0.06019682995967581
77 100 0.9375 0.7857848 0.777668 0.04841106025706223 0.0608356290174472
78 0 0.96875 0.77887464 0.78656125 0.047324179604534986 0.06069651741293532
78 50 1.0 0.76999015 0.7756917 0.04842103163919551 0.057057057057057055
78 100 0.9375 0.78183615 0.77964425 0.04695304695304695 0.06057611690022757
79 0 0.96875 0.78183615 0.7855731 0.04705

113 50 1.0 0.8084896 0.7964427 0.04223877471656346 0.04885343968095713
113 100 1.0 0.8203356 0.80731225 0.04143109276376799 0.04764043598035693
114 0 1.0 0.8094768 0.80533594 0.04045389731470679 0.052481335090030744
114 50 1.0 0.8124383 0.80237156 0.041791044776119404 0.04940711462450593
114 100 0.984375 0.8183613 0.798419 0.043754424800821645 0.04970178926441352
115 0 1.0 0.8084896 0.81422925 0.04126157926750227 0.05043518185810676
115 50 1.0 0.8084896 0.80237156 0.04054363975390629 0.050205613446720164
115 100 1.0 0.81934845 0.8033597 0.04444245016801779 0.050200803212851405
116 0 1.0 0.80256665 0.8063241 0.040877367896311065 0.0485587096259033
116 50 1.0 0.8134255 0.80039525 0.04273734382322734 0.04909819639278557
116 100 1.0 0.816387 0.80731225 0.042786069651741296 0.05005005005005005
117 0 1.0 0.80552816 0.81225294 0.04083665338645418 0.049850448654037885
117 50 1.0 0.81441265 0.8083004 0.04378109452736319 0.05039525691699605
117 100 1.0 0.8183613 0.8083004 0.04134135032456849 0.0

152 50 1.0 0.8183613 0.8201581 0.039761431411530816 0.049850448654037885
152 100 1.0 0.8203356 0.8181818 0.03788634097706879 0.04780876494023904
153 0 1.0 0.8173741 0.82114625 0.038922155688622756 0.0478799856270212
153 50 1.0 0.81934845 0.81620556 0.03988035892323031 0.049
153 100 1.0 0.816387 0.8181818 0.03792415169660679 0.05197229209086916
154 0 1.0 0.8134255 0.8241107 0.039 0.049357208448117536
154 50 1.0 0.81934845 0.80928856 0.03942684495497921 0.04826925380285064
154 100 1.0 0.82625866 0.8181818 0.036926147704590816 0.04790419161676647
155 0 1.0 0.8203356 0.8231225 0.03807615230460922 0.04885343968095713
155 50 1.0 0.81934845 0.8112648 0.04053366837177301 0.04674212480536591
155 100 1.0 0.8292201 0.80928856 0.04100232333203705 0.04604604604604605
156 0 1.0 0.8173741 0.8132411 0.03996003996003996 0.047460773745358724
156 50 1.0 0.8252715 0.8112648 0.03996003996003996 0.046841937158142694
156 100 1.0 0.82329714 0.8112648 0.03933710251577971 0.04436659080927856
157 0 1.0 0.8193484

...

345 50 1.0 0.83218163 0.8379447 0.03482587064676617 0.04187437686939183
345 100 1.0 0.8331688 0.84090906 0.03296703296703297 0.04473161033797217
346 0 1.0 0.8331688 0.84189725 0.03233719225821891 0.045614245218988304
346 50 1.0 0.8361303 0.8359684 0.033753128521144315 0.045923663512596316
346 100 1.0 0.83514315 0.8379447 0.03187250996015936 0.04601349463009542
347 0 1.0 0.83218163 0.8369565 0.03582089552238806 0.0448456901026071
347 50 1.0 0.83514315 0.83498025 0.03475670307845084 0.04386839481555334
347 100 1.0 0.8361303 0.8320158 0.035837147386999314 0.04453627180899908
348 0 1.0 0.8302073 0.8379447 0.03432149730274113 0.04419690980955803
348 50 1.0 0.83119446 0.83498025 0.03293413173652695 0.04321874875234559
348 100 1.0 0.8371175 0.8339921 0.033763654419066536 0.04395604395604396
349 0 1.0 0.8331688 0.83498025 0.03365341469981154 0.04595404595404595
349 50 1.0 0.834156 0.83300394 0.03681592039800995 0.04360801692817503
349 100 1.0 0.83218163 0.8290514 0.03781094527363184 0.04355811

384 0 1.0 0.83119446 0.8250988 0.03788634097706879 0.04386839481555334
384 50 1.0 0.8302073 0.8300395 0.036852589641434265 0.0448654037886341
384 100 1.0 0.82625866 0.8310277 0.03681592039800995 0.044044044044044044
385 0 1.0 0.8252715 0.83300394 0.03614457831325301 0.042743538767395624
385 50 1.0 0.8292201 0.8310277 0.03482587064676617 0.0437375745526839
385 100 1.0 0.83119446 0.83300394 0.03301524624328178 0.04291417165668663
386 0 1.0 0.83218163 0.8369565 0.03405226998514264 0.046073382041761486
386 50 1.0 0.83218163 0.8389328 0.03450098218114013 0.04491017964071856
386 100 1.0 0.83119446 0.8379447 0.03466052429527257 0.044326665868167844
387 0 1.0 0.82823294 0.84189725 0.034600696002472904 0.04306903022318042
387 50 1.0 0.8292201 0.8300395 0.03413654618473896 0.0454345829839901
387 100 1.0 0.8331688 0.8320158 0.031170540548625444 0.044820717131474105
388 0 1.0 0.8292201 0.8310277 0.03436138283127424 0.04514512716093744
388 50 1.0 0.83218163 0.8310277 0.03335427323581321 0.044626102

422 50 1.0 0.8292201 0.8369565 0.03465055291313929 0.04365079365079365
422 100 1.0 0.8302073 0.83992094 0.03290129611166501 0.04572564612326044
423 0 1.0 0.8361303 0.8448617 0.03314487421101439 0.04651255639397932
423 50 1.0 0.8361303 0.84090906 0.03266624786861707 0.045771144278606964
423 100 1.0 0.8331688 0.83992094 0.03139988233769083 0.043398410987343795
424 0 1.0 0.83514315 0.84090906 0.03214773599768664 0.045604263983710626
424 50 1.0 0.834156 0.8369565 0.031936127744510975 0.04621311933564898
424 100 1.0 0.8361303 0.83498025 0.03396603396603397 0.04648261268814628
425 0 1.0 0.83119446 0.8290514 0.03396603396603397 0.045554357807322234
425 50 1.0 0.8331688 0.83300394 0.03489531405782652 0.04651255639397932
425 100 1.0 0.8331688 0.83300394 0.03471038120593895 0.047
426 0 1.0 0.8331688 0.8320158 0.03589697567979898 0.04488561504371781
426 50 1.0 0.8302073 0.8290514 0.036774457307527396 0.044406515750389265
426 100 1.0 0.8252715 0.8270751 0.03423175486354164 0.045771144278606964
427

461 0 1.0 0.84007895 0.83498025 0.03339415876434633 0.044326665868167844
461 50 1.0 0.8361303 0.83992094 0.034351411449140966 0.04404719128039286
461 100 1.0 0.84007895 0.8389328 0.03571749080139998 0.04198107557791352
462 0 1.0 0.84007895 0.8379447 0.03421181209927508 0.04504504504504504
462 50 1.0 0.83810467 0.8389328 0.03177879485875537 0.04178145087235996
462 100 1.0 0.83810467 0.8379447 0.03482587064676617 0.042649818341517945
463 0 1.0 0.8361303 0.8379447 0.0338927278710102 0.04341837345789915
463 50 1.0 0.8371175 0.8389328 0.034391296977674075 0.0423104563420769
463 100 1.0 0.8371175 0.8369565 0.03353375811421221 0.043043043043043044
464 0 1.0 0.83514315 0.8359684 0.032596448193684124 0.0433485048109554
464 50 1.0 0.8361303 0.8359684 0.031250311605691666 0.041432107637641234
464 100 1.0 0.84007895 0.8379447 0.03224744981901941 0.04337844851678844
465 0 1.0 0.8371175 0.8339921 0.02896686509717112 0.0448654037886341
465 50 1.0 0.83218163 0.8369565 0.03178876624088865 0.04296921787

KeyboardInterrupt: 